In [1]:
import time
import torch
from torch import nn
#from torch.optim import Adam
from torch.optim import SGD
from torchtext.data import Field, BucketIterator


In [2]:
from torchtext.datasets import SequenceTaggingDataset

In [3]:
import csv

In [4]:
import pandas as pd

import numpy as np 

data_file = pd.read_csv('train', quoting=csv.QUOTE_NONE,sep=" ", names= ['index', 'word', 'tag'])

In [5]:
data_file

,index,word,tag
0,1,EU,B-ORG
1,2,rejects,O
2,3,German,B-MISC
3,4,call,O
4,5,to,O
...,...,...,...
204562,1,Swansea,B-ORG
204563,2,1,O
204564,3,Lincoln,B-ORG
204565,4,2,O


In [6]:
test_q = data_file[data_file['word'].apply(lambda x: len(str(x)) >= 20)]

In [7]:
#test_q

In [8]:
# convert 
data_array = data_file.to_numpy()

In [9]:
data_array

array([[1, 'EU', 'B-ORG'],
       [2, 'rejects', 'O'],
       [3, 'German', 'B-MISC'],
       ...,
       [3, 'Lincoln', 'B-ORG'],
       [4, '2', 'O'],
       [1, '-DOCSTART-', 'O']], dtype=object)

In [10]:
### store the lists of sentences into a big list 
 
def data_to_stream(data):
    data_stream = []
    i = 0 
    for index, word, tag in data:
        if index == 1:
            temp = []
            temp.append([index,word, tag])
            
        else:
            temp.append([index ,word,tag ])
            
        if ( (i+1 < len(data)) and data[i+1][0] == 1 ) or (i == len(data)-1):
            data_stream.append(temp)
        
        i += 1
            
    return data_stream 

In [11]:
data_stream = data_to_stream(data_array)

In [12]:
def write_tsv(file_name, data):
    
    with open(file_name,'w') as f:
        for i in range(len(data)):
            if i != 0:
                f.write('\n')
            for (index, w, t) in data[i]:
                #f.write(str(i))
                #f.write('\t')
                f.write(str(w))
                f.write('\t')
                f.write(str(t))
                f.write('\n')

In [13]:
dev_file = pd.read_csv('dev', quoting=csv.QUOTE_NONE, sep=" ", names= ['index', 'word', 'tag'])

In [14]:
#dev_file
# convert 
dev_array = dev_file.to_numpy()
dev_stream = data_to_stream(dev_array)


In [15]:
write_tsv('train_out.tsv',data_stream)
write_tsv('dev_out.tsv',dev_stream)


In [16]:
words = list(set(data_file["word"].values))
tags = list(set(data_file["tag"].values))

# Converting words to numbers and numbers to word
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}


In [17]:
class Corpus(object):

    def __init__(self, input_folder, min_word_freq, batch_size):
        # list all the fields
        #self.w_index = Field()
        self.word_field = Field(lower=True)
        self.tag_field = Field(unk_token=None)
        
        # create dataset using built-in parser from torchtext
        self.train_dataset, self.val_dataset = SequenceTaggingDataset.splits(
            path=input_folder,
            train="train_out.tsv",
            validation="dev_out.tsv",
            fields=(("word", self.word_field), ("tag", self.tag_field))
        )
        # convert fields to vocabulary list
        self.word_field.build_vocab(self.train_dataset, min_freq=min_word_freq)
        self.tag_field.build_vocab(self.train_dataset)
        # create iterator for batch input
        self.train_iter, self.val_iter = BucketIterator.splits(
            datasets=(self.train_dataset, self.val_dataset),
            batch_size=batch_size
        )
        # prepare padding index to be ignored during model training/evaluation
        self.word_pad_idx = self.word_field.vocab.stoi[self.word_field.pad_token]
        self.tag_pad_idx = self.tag_field.vocab.stoi[self.tag_field.pad_token]

In [18]:
corpus = Corpus(
    input_folder=".",
    min_word_freq=2,  # any words occurring less than 3 times will be ignored from vocab
    batch_size=32
)

In [19]:


#vectors = "glove.6B.100d",
                 

In [20]:
print(f"Train set: {len(corpus.train_dataset)} sentences")
print(f"dev set: {len(corpus.val_dataset)} sentences")
#print(vars(pos_dataset.examples[0]))

Train set: 14987 sentences
dev set: 3466 sentences


In [21]:
print(vars(corpus.train_dataset[0]))

{'word': ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], 'tag': ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']}


In [68]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.elu = nn.ELU()
        
    def forward(self, text):
        #pass text through embedding layer
        embedded = self.dropout(self.embedding(text))
        
        #pass embeddings into LSTM
        outputs, (hidden, cell) = self.lstm(embedded)
        
        #we use our outputs to make a prediction of what the tag should be
        outputs = self.elu(outputs)
        outputs = self.fc(self.dropout(outputs))
    
        
        return outputs

In [69]:
INPUT_DIM = len(corpus.word_field.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 128
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.33
PAD_IDX = corpus.word_pad_idx

model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [70]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

In [71]:
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(10952, 100, padding_idx=1)
  (lstm): LSTM(100, 256, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=1.0)
)

In [72]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,894,048 trainable parameters


In [73]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [74]:
#print(model.embedding.weight.data)

In [75]:
### define optimizer fuction 
optimizer = SGD(model.parameters(), lr=0.05)

In [76]:
### define loss function 
TAG_PAD_IDX = corpus.tag_field.vocab.stoi[corpus.tag_field.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [77]:
### train the model 
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.word
        tags = batch.tag
        
        optimizer.zero_grad()
        
        predictions = model(text)
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [78]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.word
            tags = batch.tag
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [79]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

In [80]:
N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, corpus.train_iter, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, corpus.val_iter, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 43s
	Train Loss: 0.894 | Train Acc: 82.99%
	 Val. Loss: 1.493 |  Val. Acc: 78.46%
Epoch: 02 | Epoch Time: 1m 38s
	Train Loss: 0.772 | Train Acc: 83.23%
	 Val. Loss: 1.351 |  Val. Acc: 78.46%


In [35]:
#model.load_state_dict(torch.load('tut1-model.pt'))

#index = 0
dev_sentences = []
dev_actual_tags = []
for i in range(len(corpus.val_dataset)):
    
    sentence_words = vars(corpus.val_dataset[i])['word']
    dev_sentences.append(sentence_words)
    actual_tag = vars(corpus.val_dataset[i])['tag']
    dev_actual_tags.append(actual_tag)

    


In [36]:
#sentence_tag
#dev_sentences

In [43]:
def tag_sentence(model, device, sentence, text_field, tag_field):
    
    model.eval()
    #tokens = [token for token in sentence]

    tokens = sentence
        
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    #print(top_predictions)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [45]:
#from seqeval.metrics import classification_report, f1_score

In [46]:
#dev_sentences[1]

In [47]:
whole_pred_tags = []

for s in dev_sentences:
    
    _, pred_tags, _ = tag_sentence(model, device, s, corpus.word_field, corpus.tag_field)
    
    whole_pred_tags.append(pred_tags)
    
    

In [48]:
s_1 = ['london', '1996-08-30']
_,pred_t, _ = tag_sentence(model, device, s_1, corpus.word_field, corpus.tag_field)

In [49]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [50]:
whole_pred_tags_f = flatten(whole_pred_tags)

In [51]:
dev_actual_tags_f = flatten(dev_actual_tags)

In [52]:
#whole_pred_tags_f

In [53]:
from sklearn.metrics import classification_report
print(classification_report(dev_actual_tags_f, whole_pred_tags_f))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00      1837
      B-MISC       0.00      0.00      0.00       922
       B-ORG       0.00      0.00      0.00      1341
       B-PER       0.00      0.00      0.00      1842
       I-LOC       0.00      0.00      0.00       257
      I-MISC       0.00      0.00      0.00       346
       I-ORG       0.00      0.00      0.00       751
       I-PER       0.00      0.00      0.00      1307
           O       0.83      1.00      0.91     42975

    accuracy                           0.83     51578
   macro avg       0.09      0.11      0.10     51578
weighted avg       0.69      0.83      0.76     51578



/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
#corpus.tag_field.vocab.itos[.item()

In [56]:
import torchtext.vocab as vocab


custom_embeddings = vocab.Vectors(name = 'glove.6B.100d')

100%|█████████▉| 399999/400000 [00:18<00:00, 21875.81it/s]


In [62]:
import torchtext.vocab as vocab


In [85]:

word_field = Field()
tag_field = Field(unk_token=None)
train_dataset,val_dataset = SequenceTaggingDataset.splits(
path='.',
train="train_out.tsv",
validation="dev_out.tsv",
fields=(("word",word_field), ("tag",tag_field)))
word_field.build_vocab(train_dataset, min_freq=1)
tag_field.build_vocab(train_dataset)
# create iterator for batch input
train_iter, val_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset),
    batch_size=32
)

In [86]:
word_field.build_vocab(train_dataset, vectors = custom_embeddings)
word_pad_idx = word_field.vocab.stoi[word_field.pad_token]
tag_pad_idx = tag_field.vocab.stoi[tag_field.pad_token]

In [89]:
INPUT_DIM = len(word_field.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 128
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.33
PAD_IDX = corpus.word_pad_idx

model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

In [90]:
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(23626, 100, padding_idx=1)
  (lstm): LSTM(100, 256, bidirectional=True)
  (fc): Linear(in_features=512, out_features=128, bias=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (elu): ELU(alpha=1.0)
)

In [99]:
model.embedding.weight.data.copy_(word_field.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.0823,  0.3968,  0.6859,  ...,  0.0866, -0.0148, -0.1637],
        [-0.3110, -0.3398,  1.0308,  ...,  0.5317,  0.2836, -0.0640],
        [-0.5832, -0.5807,  0.3504,  ...,  0.4043, -0.0192,  0.0945]])

In [100]:
optimizer = SGD(model.parameters(), lr=0.05)

In [102]:
TAG_PAD_IDX = tag_field.vocab.stoi[tag_field.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [104]:
N_EPOCHS = 8

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 52s
	Train Loss: 0.472 | Train Acc: 84.97%
	 Val. Loss: 1.040 |  Val. Acc: 77.77%
Epoch: 02 | Epoch Time: 2m 18s
	Train Loss: 0.441 | Train Acc: 86.06%
	 Val. Loss: 0.992 |  Val. Acc: 78.34%
Epoch: 03 | Epoch Time: 1m 33s
	Train Loss: 0.417 | Train Acc: 86.86%
	 Val. Loss: 0.951 |  Val. Acc: 78.95%
Epoch: 04 | Epoch Time: 1m 46s
	Train Loss: 0.398 | Train Acc: 87.42%
	 Val. Loss: 0.920 |  Val. Acc: 79.33%
Epoch: 05 | Epoch Time: 1m 47s
	Train Loss: 0.380 | Train Acc: 87.98%
	 Val. Loss: 0.893 |  Val. Acc: 79.74%
Epoch: 06 | Epoch Time: 1m 38s
	Train Loss: 0.364 | Train Acc: 88.37%
	 Val. Loss: 0.860 |  Val. Acc: 80.69%
Epoch: 07 | Epoch Time: 1m 35s
	Train Loss: 0.351 | Train Acc: 88.74%
	 Val. Loss: 0.839 |  Val. Acc: 81.02%
Epoch: 08 | Epoch Time: 1m 42s
	Train Loss: 0.339 | Train Acc: 89.09%
	 Val. Loss: 0.814 |  Val. Acc: 81.41%


In [105]:


model.load_state_dict(torch.load('blstm1_test.pt', map_location=torch.device('cpu')))

RuntimeError: Error(s) in loading state_dict for BiLSTMPOSTagger:
	size mismatch for embedding.weight: copying a param with shape torch.Size([7519, 100]) from checkpoint, the shape in current model is torch.Size([23626, 100]).